In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from itertools import product
import collections
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import time
import pandas as pd
import json
from IPython.display import clear_output
import import_dataset as load
from torch.utils.data import Dataset, DataLoader

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)



In [2]:
#Importation du data
train_path = "../dataset/train" 
test_path = "../dataset/test" 

train_dataset_X, train_dataset_Y = load.loadDataset4D(train_path)
test_dataset_X, test_dataset_Y = load.loadDataset4D(test_path)

print(train_dataset_X.shape)
print(train_dataset_Y.shape)

debut chargement image
debut chargement image
(318, 1, 224, 224)
(318,)


In [3]:
# Preprocessing on Y labels

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

train_dataset_Y = encoder.fit_transform(train_dataset_Y)
test_dataset_Y = encoder.fit_transform(test_dataset_Y)

In [4]:
# FOR 2D ONLY 

# train_df_X = pd.DataFrame(train_dataset_X)
# train_df_Y = pd.DataFrame(train_dataset_Y)
# test_df_X = pd.DataFrame(test_dataset_X)
# test_df_Y = pd.DataFrame(test_dataset_Y)

# train_df_Y.head()

# FOR 4D ONLY 
# train_dataset_X_conv = np.ones([12000, 1, 224, 224])


# print(train_dataset_X_conv.shape)

In [5]:
#Create custom dataLoader

class DogsDataset(Dataset):
    """ Dogs dataset."""
    # Initialize your data, download, etc.
    def __init__(self, X, Y):
        self.len = Y.shape[0]
        self.x_data = torch.from_numpy(X).float()
        self.y_data = torch.from_numpy(Y).float()
        print(self.x_data.shape)
        print(self.y_data.shape)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


dataset_train = DogsDataset(train_dataset_X, train_dataset_Y)
dataset_test = DogsDataset(test_dataset_X, test_dataset_Y)

train_loader = DataLoader(dataset=dataset_train,
                          batch_size=32,
                          shuffle=True,
                          num_workers=0)


torch.Size([318, 1, 224, 224])
torch.Size([318])
torch.Size([318, 1, 224, 224])
torch.Size([318])


In [6]:
#create loaders: 

#trainloaderX = DataLoader(train_df_X, batch_size=4, shuffle=True, num_workers=2)
#testloaderX = DataLoader(test_df_X, batch_size=4, shuffle=True, num_workers=2)



In [27]:
# Réseau à 4 couches
class ModelCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # On défini nos couches:
    
        #Premiere couche de convolution
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fonction1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fonction2 = nn.Linear(in_features=120, out_features=60)
        self.output = nn.Linear(in_features=60, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = x.reshape(-1, 12*4*4) # surement important adapter à la size de l'image 
        x = self.fonction1(x)
        x = nn.functional.relu(x)
        x = self.fonction2(x)
        x = nn.functional.relu(x)
        # output
        x = self.output(x)
        #x = nn.Softmax(x)
        return x

In [28]:
#instance du model 

model = ModelCNN()

# Définir la fonction de loss : 

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) #Essayer avec Adam aussi

In [29]:
#Training the network 

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
#         print("on est passés dans la boucle :")
#         print(data)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        print(inputs.shape)
        outputs = model(inputs)
        
        
        print(outputs.shape)
        print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

torch.Size([32, 1, 224, 224])


AttributeError: 'Softmax' object has no attribute 'shape'

In [ ]:
# Modifier les keyword des images de test pour les retrouver

from iptcinfo3 import IPTCInfo
info = IPTCInfo('oiseau.jpg')
info['keywords'] = ["key1",  "key2"]
info.save()
